## This notebook provides some useful tests to assess the quality of the signals acquired by the python-AcqBIT application (name is a work in progress).

In [1]:
import numpy as np
import h5py as h5
import os

In [2]:
# Open Configuration file
macAddress = '20:16:04:12:01:23'

# Fetch created directory
path_name = os.path.join('~', 'Desktop', 'test', macAddress.replace(':', '-'))   
user = os.path.expanduser(path_name)
path_to_save = os.path.expanduser(path_name)

In [3]:
# Choose the file to verify
f_name_list = os.listdir(path_to_save)
f_name = f_name_list[0]
file_path_name = os.path.join(path_to_save, f_name)

print file_path_name
with h5.File(file_path_name, 'r') as f:
    print f

C:\Users\franciscosargo\Desktop\test\20-16-04-12-01-23\Sun, 13 Jan 2019 13 42 52 +0000.h5


IOError: Unable to open file (bad object header version number)